## Modelo 4 - Experimentando modelos em todos os dados

(arquivo ./datasets/raw_data_with_all_labels.xlsx)

In [1]:
#importando pacotes
import pandas as pd
import numpy as np
import re
import time

import bs4
import json

import glob
import tqdm

pd.set_option("max.columns", 100)

#https://strftime.org
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
df = pd.read_excel('./datasets/raw_data_with_all_labels.xlsx', index_col=0).dropna(subset=["y"])

In [6]:
df.head()

,watch-title,y,watch-view-count,view,watch-time-text,watch7-headline,watch8-sentiment-actions,og:image,og:image:width,og:description,og:video:width,og:video:height,og:video:tag,content_watch-info-tag-list,channel_link_0
0,Finanças é Coisa de Criança!,0.0,635 visualizações,NaN,Publicado em 30 de jul. de 2019,#intusforma #institucional #educacao\n\n\n\n ...,635 visualizações\n\n\n\n\nGostou deste vídeo?...,https://i.ytimg.com/vi/-0HZcKmHXn4/maxresdefau...,1280,Um dos nossos primeiros programas que deram vi...,1280.0,720.0,financasecoisadecrianca,Educação,/channel/UCjE76zhvoDw5hnLtmzFHR-Q
1,Câncer 2020 - Profissional e Finanças 1° semestre,0.0,37.305 visualizações,NaN,Publicado em 3 de dez. de 2019,#tarô #previsão2020 #profissional2020\n\n\n\n ...,37.305 visualizações\n\n\n\n\n\n\n\n2.550\n\nG...,https://i.ytimg.com/vi/-1kS-ZrfcYQ/hqdefault.jpg,480,Previsão primeiro semestre de 2020 profissiona...,1280.0,720.0,tarot câncer 2020 profissional,Pessoas e blogs,/channel/UCL6feH5A7tjCoNTKAkJ9ITQ
2,Seja Rica: Conquiste sua Independência Financeira,0.0,98 visualizações,NaN,Publicado em 30 de mar. de 2017,Seja Rica: Conquiste sua Independência Financeira,98 visualizações\n\n\n\n\n\n\n\n29\n\nGostou d...,https://i.ytimg.com/vi/-40s-kunvQM/hqdefault.jpg,480,Seja Rica é o novo quadro aqui do canal. Nesse...,640.0,360.0,Rico,Pessoas e blogs,/channel/UChgFe4WPc8Wx4tJVAdd9-rA
3,Independência FINANCEIRA,0.0,41 visualizações,NaN,Publicado em 19 de mar. de 2019,Independência FINANCEIRA,41 visualizações\n\n\n\n\n\n\n\n17\n\nGostou d...,https://i.ytimg.com/vi/-6dz-10LkHc/maxresdefau...,1280,"Como conseguir sua independência financeira, m...",1280.0,720.0,pouco,Pessoas e blogs,/channel/UCUH-FVOIrY6IW2hs66A4IFA
5,6 ERROS para NÃO ter a INDEPENDÊNCIA FINANCEIR...,0.0,389 visualizações,NaN,Publicado em 21 de jan. de 2020,6 ERROS para NÃO ter a INDEPENDÊNCIA FINANCEIR...,389 visualizações\n\n\n\n\n\n\n\n86\n\nGostou ...,https://i.ytimg.com/vi/-CcyxsGoquk/maxresdefau...,1280,"Ano novo, vida financeira nova! A partir de ag...",1280.0,720.0,não ter independência financeira,Educação,/channel/UCu8nmn2Na1wcPUxy7dDqG2A


In [5]:
df.duplicated().mean()

0.0

In [7]:
df.duplicated(['watch-title']).mean()

0.0017667844522968198

In [12]:
df = df[~df.duplicated(['watch-title'], keep=False)]

In [14]:
df.duplicated(['watch-title']).mean()

0.0

In [13]:
df.shape

(564, 15)

In [16]:
#criando dataframe com os índices do df_labeled
df_limpo = pd.DataFrame(index = df.index)

In [19]:
df_limpo['title'] = df['watch-title']

## Limpando os dados

In [24]:
#formato da data de publicação no dataframe com labels:
df['watch-time-text'].head()

0    Publicado em 30 de jul. de 2019
1     Publicado em 3 de dez. de 2019
2    Publicado em 30 de mar. de 2017
3    Publicado em 19 de mar. de 2019
5    Publicado em 21 de jan. de 2020
Name: watch-time-text, dtype: object

In [20]:
map_mes = {'jan.':'Jan',
             'fev.':'Feb',
              'mar.':'Mar',
              'abr.':'Apr',
              'mai.':'May',
              'jun.':'Jun',
              'jul.':'Jul',
              'ago.':'Aug',
              'set.':'Sep',
              'out.':'Oct',
              'nov.':'Nov',
              'dez.':'Dec'}

In [21]:
def limpa_data (row):
    
    data_limpa = re.search(r"(\d+) de ([a-z]+)\. de (\d+)", row.loc['watch-time-text']).group()
    data_limpa = data_limpa.split('de ')
    data_limpa = [x.strip() for x in data_limpa]
    data_limpa[0] = ['0' + str(data_limpa[0]) if len(data_limpa[0]) == 1 else data_limpa[0]][0]
    data_limpa[1] = [v for k, v in map_mes.items() if k == data_limpa[1]][0]
    data = '-'.join(data_limpa)

    return data

In [25]:
df['data'] = df.apply(limpa_data, axis=1)

d:\solucao completa de ds - mario filho\projeto\.env\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [28]:
df_limpo['data'] = pd.to_datetime(df['data']).copy()

In [29]:
#coluna com informação de views
df['watch-view-count'].head()

0       635 visualizações
1    37.305 visualizações
2        98 visualizações
3        41 visualizações
5       389 visualizações
Name: watch-view-count, dtype: object

In [32]:
#adicionando coluna com view count formatado
df_limpo['views'] = df['watch-view-count'].str.extract(r"(\d+\.?\d*)", expand=False).str.replace(".","").fillna(0).astype(int)

## Criando Features

In [33]:
df_limpo.head()

,title,data,views
0,Finanças é Coisa de Criança!,2019-07-30,635
1,Câncer 2020 - Profissional e Finanças 1° semestre,2019-12-03,37305
2,Seja Rica: Conquiste sua Independência Financeira,2017-03-30,98
3,Independência FINANCEIRA,2019-03-19,41
5,6 ERROS para NÃO ter a INDEPENDÊNCIA FINANCEIR...,2020-01-21,389


In [34]:
#criando dataframe para as features:
features = pd.DataFrame(index=df_limpo.index)
#criando série com as labels
y = df['y'].copy()

In [35]:
#criando coluna com dias que se passaram desde a publicação até a data referência
features['dias_publicado'] = (pd.to_datetime("2020-05-08") - df_limpo['data']) / np.timedelta64(1, 'D') #denominador cria um objeto timedelta do numpy em diferença de 1 dia

#adicionando a coluna de views ao dataframe features
features['views'] = df_limpo['views']

#calculando views por dia
features['views_por_dia'] = features['views']/features['dias_publicado']

# retirando coluna de dias após publicação (para explicação, ver notebook para modelo1)
features = features.drop('dias_publicado', axis = 1)

In [37]:
features.head()

,views,views_por_dia
0,635,2.243816
1,37305,237.611465
2,98,0.086344
3,41,0.098558
5,389,3.601852


In [38]:
#criando sets de treino e validação, separando os dados em 50% (quantil 0.5)
mask_train = df_limpo.data < df_limpo.data.quantile(0.5)
mask_val = df_limpo.data >= df_limpo.data.quantile(0.5)

Xtrain, Xval = features[mask_train], features[mask_val]
ytrain, yval = y[mask_train], y[mask_val]
Xtrain.shape, Xval.shape, ytrain.shape, yval.shape

((282, 2), (282, 2), (282,), (282,))

In [88]:
from sklearn.feature_extraction.text import TfidfVectorizer

title_train = df_limpo[mask_train]['title'] #coletando títulos do set de treino
title_val = df_limpo[mask_val]['title'] #coletando títulos do set de validação

title_vec = TfidfVectorizer(min_df = 2, ngram_range=(1,3)) #vetorizador. lembrar que modelos não trabalham com strings
#ocorrência mínima do termo em 2 vídeos

title_bow_train = title_vec.fit_transform(title_train)
title_bow_val = title_vec.transform(title_val) #usamos somente transform para transformar somente baseado nas palavras no dataset passado para ele.

In [89]:
title_bow_train.shape  

(282, 497)

Mais sobre Tf iDF [aqui](https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction).
O resultado acima indica que foram criadas 497 colunas no title_bow_train. Cada coluna é um vetor para cada palavra do título. O formato será uma matriz esparsa (só armazena valores maiores que zero), do scipy.

In [90]:
title_bow_train

<282x497 sparse matrix of type '<class 'numpy.float64'>'
	with 2584 stored elements in Compressed Sparse Row format>

A matriz criada para title_bow_train armazenou 1411 valores na mmemória. Se fosse armazenar todos os valores, teríamos:

Para juntar as features de texto com as numéricas, deve-se usar a função hstack do scipy.sparse (hstack e vstack permitem lidar com matrizes esparsas com mais eficiência.
Vamos juntas as features para poder treinar o novo modelo. Mais sobre scipy sparse [aqui](https://docs.scipy.org/doc/scipy/reference/sparse.html).

In [91]:
from scipy.sparse import hstack, vstack

In [92]:
Xtrain_wtitle = hstack([Xtrain, title_bow_train]) #aqui, juntamos as features de treino em Xtrain com as geradas para os títulos das features de treino
Xval_wtitle = hstack([Xval, title_bow_val]) #ídem para validação

In [93]:
# o total de colunas será a soma das colunas das features numéricas com as de texto (2 + 211 = 213)
Xtrain_wtitle.shape, Xval_wtitle.shape

((282, 499), (282, 499))

In [94]:
#importando alguns modelos
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [95]:
%%time
#treinando modelo
# 1000 árvores

mdl = RandomForestClassifier(n_estimators = 1000, min_samples_leaf=1, random_state = 0, class_weight="balanced", n_jobs=8)
mdl.fit(Xtrain_wtitle, ytrain)

Wall time: 1.65 s


RandomForestClassifier(class_weight='balanced', n_estimators=1000, n_jobs=8,
                       random_state=0)

## Calculando as novas métricas de referência

In [96]:
p = mdl.predict_proba(Xval_wtitle)[:, 1]

In [97]:
from sklearn.metrics import roc_auc_score, average_precision_score

In [98]:
average_precision_score(yval, p)

0.7094600167650358

In [99]:
roc_auc_score(yval, p)

0.8830196235482579

In [100]:
metricas = pd.DataFrame([['Random Forest', average_precision_score(yval, p), roc_auc_score(yval, p) ]], columns = ['modelo', 'avg_pre', 'roc_auc'])

In [101]:
metricas

,modelo,avg_pre,roc_auc
0,Random Forest,0.70946,0.88302


## Treinando com LightGBM
Geralmente performa melhor que a random forest, mas requer tuning de mais parâmetros

In [104]:
from lightgbm import LGBMClassifier

In [105]:
mdl2 = LGBMClassifier(randomstate=0, class_weight="balanced", n_jobs=8)

In [106]:
mdl2.fit(Xtrain_wtitle, ytrain)

LGBMClassifier(class_weight='balanced', n_jobs=8, randomstate=0)

In [110]:
p2 = mdl2.predict_proba(Xval_wtitle)[:, 1]

d:\solucao completa de ds - mario filho\projeto\.env\lib\site-packages\lightgbm\basic.py:546: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')


In [111]:
average_precision_score(yval, p2)

0.6631167492370266

In [112]:
roc_auc_score(yval, p2)

0.8796155386463756

## Bayesian optimization
Passar algumas combinações de parâmetros para os modelos a fim de encontrar alguma que retorne resultado satisfatório para um espaço de tempo pequeno.

In [114]:
from skopt import forest_minimize

In [123]:
def tune_lgbm(params):
    print(params)
    lr=params[0]
    max_depth=params[1]
    min_child_samples=params[2]
    subsample=params[3] #pega uma amostra dos dados a cada rodada de treino para diminuir overfitting
    colsample_bytree=params[4]
    n_estimators=params[5]
    
    min_df=params[6]
    ngram_range=(1, params[7])
    
    title_vec = TfidfVectorizer(min_df = min_df, ngram_range=ngram_range) 
    title_bow_train = title_vec.fit_transform(title_train)
    title_bow_val = title_vec.transform(title_val) 
    
    Xtrain_wtitle = hstack([Xtrain, title_bow_train]) 
    Xval_wtitle = hstack([Xval, title_bow_val])
    
    mdl =LGBMClassifier(learning_rate=lr, num_leaves=2 ** max_depth, max_depth = max_depth, min_child_samples = min_child_samples,
                       subsample = subsample, colsample_bytree = colsample_bytree, bagging_freq = 1, n_estimators = n_estimators,
                       random_state=0, class_weight = 'balanced', n_jobs = 8)
    
    mdl.fit(Xtrain_wtitle, ytrain)
    
    p = mdl.predict_proba(Xval_wtitle)[:, 1]
    
    print(roc_auc_score(yval, p))
    
    return -average_precision_score(yval, p) #o forest_minimize vai minizar a métrica. Logo, para maximizar, vamos passar a negativa



In [125]:
space =[(1e-3, 1e-1, 'log-uniform'), #lr #log-uniform para pegar com mais chances números mais próximos de 1e-3.
       (1, 10), #max_depth
       (1, 20), #min_child_sample
       (0.05, 1.), #subsample
       (0.05, 1.), #colsample_bytree
       (100, 1000), #n_estimators
       (1, 5), #min_df
       (1, 5)] #ngram_range


 

In [126]:
res = forest_minimize(tune_lgbm, space, random_state = 160745, n_random_starts = 20, n_calls = 50, verbose = 1)
   

Iteration No: 1 started. Evaluating function at random point.
[0.009944912110647982, 5, 1, 0.4677107511929402, 0.49263223036174764, 272, 3, 1]
0.8595915098117741
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.1260
Function value obtained: -0.6669
Current minimum: -0.6669
Iteration No: 2 started. Evaluating function at random point.
[0.053887464791860025, 1, 15, 0.7437489153990157, 0.8675167974293533, 549, 3, 4]


d:\solucao completa de ds - mario filho\projeto\.env\lib\site-packages\lightgbm\basic.py:546: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')
d:\solucao completa de ds - mario filho\projeto\.env\lib\site-packages\lightgbm\basic.py:546: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')
d:\solucao completa de ds - mario filho\projeto\.env\lib\site-packages\lightgbm\basic.py:546: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')


0.8478173808570285
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 0.0908
Function value obtained: -0.6188
Current minimum: -0.6669
Iteration No: 3 started. Evaluating function at random point.
[0.004151454520895999, 6, 20, 0.8682075103820793, 0.9491436163200662, 411, 4, 3]
0.8656387665198239
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 0.1067
Function value obtained: -0.6888
Current minimum: -0.6888
Iteration No: 4 started. Evaluating function at random point.
[0.0014099928811969545, 9, 9, 0.6502182010234373, 0.6866210554187129, 828, 5, 2]


d:\solucao completa de ds - mario filho\projeto\.env\lib\site-packages\lightgbm\basic.py:546: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')
d:\solucao completa de ds - mario filho\projeto\.env\lib\site-packages\lightgbm\basic.py:546: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')


0.877693231878254
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 0.3221
Function value obtained: -0.6964
Current minimum: -0.6964
Iteration No: 5 started. Evaluating function at random point.
[0.08530558241838007, 8, 19, 0.2137736299768322, 0.1313765544201984, 961, 4, 1]
0.7895474569483381
Iteration No: 5 ended. Evaluation done at random point.
Time taken: 0.0828
Function value obtained: -0.3957
Current minimum: -0.6964
Iteration No: 6 started. Evaluating function at random point.
[0.003567949451535685, 10, 19, 0.7232951768944309, 0.7298538828427115, 939, 4, 3]


...

0.8312374849819784
Iteration No: 48 ended. Search finished for the next optimal point.
Time taken: 0.6164
Function value obtained: -0.5772
Current minimum: -0.7197
Iteration No: 49 started. Searching for the next optimal point.
[0.047859445445345046, 10, 3, 0.19394338654139964, 0.059841131242082624, 890, 3, 5]


d:\solucao completa de ds - mario filho\projeto\.env\lib\site-packages\lightgbm\basic.py:546: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')


0.8583099719663596
Iteration No: 49 ended. Search finished for the next optimal point.
Time taken: 0.5844
Function value obtained: -0.6506
Current minimum: -0.7197
Iteration No: 50 started. Searching for the next optimal point.
[0.012984835983539925, 8, 6, 0.2844796605294211, 0.14391092638462988, 992, 5, 4]
0.8849018822587104


d:\solucao completa de ds - mario filho\projeto\.env\lib\site-packages\lightgbm\basic.py:546: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')


Iteration No: 50 ended. Search finished for the next optimal point.
Time taken: 0.5048
Function value obtained: -0.7054
Current minimum: -0.7197


In [128]:
#parâmetros que deram o melhor resultado
res.x

[0.00781968225875022, 3, 4, 0.7078936710077383, 0.31818755505678337, 275, 4, 4]

In [129]:
space

[(0.001, 0.1, 'log-uniform'),
 (1, 10),
 (1, 20),
 (0.05, 1.0),
 (0.05, 1.0),
 (100, 1000),
 (1, 5),
 (1, 5)]

In [131]:
#valor maximizado da average_precision
res.fun

-0.7197493613596355

## Logistic Regression

In [133]:
from sklearn.preprocessing import MaxAbsScaler, StandardScaler
from scipy.sparse import csr_matrix #converte matrix densa em sparse

In [134]:
Xtrain_wtitle2 = csr_matrix(Xtrain_wtitle.copy())
Xval_wtitle2 = csr_matrix(Xval_wtitle.copy())

In [147]:
scaler = StandardScaler()

In [138]:
Xtrain_wtitle2[:, :2] = scaler.fit_transform(Xtrain_wtitle2[:, :2].todense())
Xval_wtitle2[:, :2] = scaler.transform(Xval_wtitle2[:, :2].todense())

In [139]:
Xval_wtitle2.shape

(282, 499)

In [140]:
mdl3 = LogisticRegression(n_jobs=6, random_state=0)

In [141]:
mdl3.fit(Xtrain_wtitle2, ytrain)

LogisticRegression(n_jobs=6, random_state=0)

In [178]:
p3 = mdl3.predict_proba(Xval_wtitle2)[:,1]

In [179]:
average_precision_score(yval, p3)

0.5038021736634999

In [146]:
#com MaxAbs
scaler2 = MaxAbsScaler()

In [164]:
Xtrain_wtitle3 = csr_matrix(Xtrain_wtitle.copy())
Xval_wtitle3 = csr_matrix(Xval_wtitle.copy())

In [165]:
Xtrain_wtitle3 = scaler2.fit_transform(Xtrain_wtitle3)
Xval_wtitle3 = scaler2.transform(Xval_wtitle3)

In [166]:
mdl4 = LogisticRegression(n_jobs=6, random_state=0)
mdl4.fit(Xtrain_wtitle3, ytrain)

LogisticRegression(n_jobs=6, random_state=0)

In [167]:
p4 = mdl4.predict_proba(Xval_wtitle3)[:,1]

In [168]:
average_precision_score(yval, p4)

0.507682253512145

In [171]:
mdl5 = LogisticRegression(C=10., n_jobs=6, random_state=0)
mdl5.fit(Xtrain_wtitle3, ytrain)
p5 = mdl5.predict_proba(Xval_wtitle3)[:,1]
average_precision_score(yval, p5)

0.44543872441768084

In [174]:
mdl6 = LogisticRegression(C=.5, n_jobs=6, random_state=0)
mdl6.fit(Xtrain_wtitle3, ytrain)
p6 = mdl6.predict_proba(Xval_wtitle3)[:,1]
average_precision_score(yval, p6)

0.5165408985966196

In [176]:
mdl7 = LogisticRegression(C=.25, n_jobs=6, random_state=0)
mdl7.fit(Xtrain_wtitle3, ytrain)
p7 = mdl7.predict_proba(Xval_wtitle3)[:,1]
average_precision_score(yval, p7)

0.5185412819544385

In [183]:
mdl8 = LogisticRegression(C=.1, n_jobs=6, random_state=0)
mdl8.fit(Xtrain_wtitle3, ytrain)
p8 = mdl8.predict_proba(Xval_wtitle3)[:,1]
average_precision_score(yval, p8)

0.5217553251995134